<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [7]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [11]:
df = pd.read_csv('data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [14]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [18]:
import re 
def tokenize(text):
    """Parses a string into a list of semantic units (words)

    Args:
        text (str): The string that the function will tokenize.

    Returns:
        list: tokens parsed out by the mechanics of your choice
    """
    
    tokens = re.sub('[^a-zA-Z]', ' ', text)
    tokens = tokens.lower().split()
    tokens = [x.strip(' ') for x in tokens]
    
    return tokens

In [19]:
df['tokens'] = df['reviews.text'].apply(tokenize)

In [ ]:
from ast import literal_eval

# df['tokens'] = df['tokens'].apply(literal_eval)

In [21]:
import gensim
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [23]:
id2word = corpora.Dictionary(df['tokens'])

In [24]:
len(id2word.keys())

9958

In [25]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [27]:
lda = LdaMulticore(corpus=corpus, 
                  id2word=id2word,
                  num_topics=10, 
                  passes=25,
                  workers=4
                )

In [28]:
import re

words = [re.findall(r'"[^"]*"', t[1]) for t in lda.print_topics(10)]

In [29]:
topics = [', '.join(t[0:5]) for t in words]

In [30]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
"it", "for", "my", "and", "to"

------ Topic 1 ------
"the", "and", "is", "i", "for"

------ Topic 2 ------
"as", "well", "good", "just", "so"

------ Topic 3 ------
"i", "batteries", "the", "a", "these"

------ Topic 4 ------
"the", "to", "for", "is", "it"

------ Topic 5 ------
"they", "are", "them", "and", "long"

------ Topic 6 ------
"the", "to", "i", "it", "and"

------ Topic 7 ------
"i", "it", "was", "a", "the"

------ Topic 8 ------
"i", "very", "recommend", "would", "with"

------ Topic 9 ------
"great", "price", "for", "good", "a"



In [31]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [32]:
pyLDAvis.gensim.prepare(lda,corpus, id2word)

/home/cjmckay7/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.099069 -0.060871       1        1  18.995343
3      0.055882 -0.116194       2        1  18.219442
4     -0.084883  0.011810       3        1  14.409146
0     -0.185872 -0.004918       4        1  12.390279
7     -0.117540 -0.049338       5        1  10.340585
1     -0.146434 -0.013057       6        1   9.394929
9      0.102326  0.253649       7        1   4.709361
2      0.321795  0.009886       8        1   4.188504
5      0.167165 -0.203343       9        1   3.858649
8     -0.013370  0.172377      10        1   3.493763, topic_info=      Term          Freq         Total Category  logprob  loglift
43      as   7867.000000   7867.000000  Default  30.0000  30.0000
41    they   6084.000000   6084.000000  Default  29.0000  29.0000
51   great   9099.000000   9099.000000  Default  28.0000  28.0000
7        i  23331.000000  23331.000000  Default  27.0000  27.0000
40   price   5083.000000   5083.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
0        a    323.725549  17646.694439  Topic10  -4.3839  -0.6442
8       is    247.170283  11250.972900  Topic10  -4.6537  -0.4639
269    all    204.152619   2666.545323  Topic10  -4.8449   0.7845
164     it    241.284824  19117.856602  Topic10  -4.6778  -1.0182
167     my    217.936536  11232.580836  Topic10  -4.7795  -0.5882

[786 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.197544         a
0         2  0.202758         a
0         3  0.179467         a
0         4  0.102512         a
0         5  0.119512         a
...     ...       ...       ...
4963      4  0.927278  youngest
3810      3  0.796779   youtube
3810      4  0.199195   youtube
2768      4  0.998117        yr
1713      9  0.917109       yup

[1773 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 5, 1, 8, 2, 10, 3, 6, 9])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling